### Imports

In [2]:
import pandas as pd
from arcgis.gis import GIS
from arcgis.apps.storymap import JournalStoryMap
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer, Feature
from arcgis.geometry import Geometry, SpatialReference, Point

### Utility Methods

In [101]:
import uuid,json,arcpy

def getExtent(fromFeature):
    g = Geometry(fromFeature.geometry)
    if g.type.lower() == 'point':
        bg = Point({"x": g.x, "y": g.y, "spatialReference": {"wkid": 102100}})
        gbuf = bg.buffer(1609.344) # assume meters, 1609.344 ~ 1 mile
        return gbuf.extent
    
    return g.extent

def getMapAction(fromFeature, forWebmap, lyrNdx=0, label_field="OBJECTID", label_prefix="Feature Extent", add_popup=False):
    # TODO - make sure input parameters are valid, since we are just inserting them into the mapaction schema
    
    # pull out a value from the feature to display in our anchor tag
    lbl_val = fromFeature.attributes[fromFeature.fields[0]]
    if label_field in fromFeature.fields:
        lbl_val = fromFeature.attributes[label_field]
    
    # format the label
    lbl_frmtd = '{0}{1}'.format(label_prefix, lbl_val)
    
    # get the geometry extent
    # TODO - Need to validate this or default to a valid extent
    ext = getExtent(fromFeature)
    
    # find the oid field
    oid_field = [f for f in fromFeature.fields if f.lower() == 'objectid'][0]
    if len(oid_field) <= 0 or oid_field is None:
        print('objectid not in field list, will disable showing popup')
        add_popup = False
    
    wm = WebMap(forWebmap)
    layer = wm.layers[lyrNdx]
    
    popup_def = None
    if add_popup:
        popup_def = {
            "layerId": layer.id,
            "fieldName": oid_field, # we should probably get this dynamically and not hard code
            "fieldValue": fromFeature.attributes[oid_field], # same with this one
            "anchorPoint": {
                "x": Geometry(fromFeature.geometry).centroid[0],
                "y": Geometry(fromFeature.geometry).centroid[1],
                "spatialReference": {
                    "wkid": 102100
                }
            }
        }
    
    # create a unique id
    # ts = int(round(time.time() * 1000)) this isn't precise enough, gets duplicated in loop
    ts = str(uuid.uuid4())
    #print('id : {0}'.format(ts))
    
    # insert values into our map action schema
    ma = {
        "id":"MJ-ACTION-{0}".format(ts),
        "type": "media",
        "media": {
            "type":"webmap",
            "webmap": {
                "id":forWebmap.id,
                "extent": {
                    "xmin": ext[0],
                    "ymin": ext[1],
                    "xmax": ext[2],
                    "ymax": ext[3],
                    "spatialReference":{
                        "wkid": 102100
                    }
                },
                "layers": None,
                "popup": popup_def,
                "overview":{
                    "enable": False,
                    "openByDefault": False
                },
                "legend": {
                    "enable": False,
                    "openByDefault": False
                },
                "geocoder":{
                    "enable": False
                },
                "altText": ""
            }
        }
    }
    
    # create the html string
    content_anchor = '<a data-storymaps="{0}" data-storymaps-type="media">{1}</a>'.format(ma['id'], lbl_frmtd)
    
    # hand back our results to whoever called this function
    return {"content_link": content_anchor, "map_action": ma}

### Connect to Portal

In [7]:
prtl = GIS(username='marktorrey')

Enter password: ········


### Get a webmap that has some data we want to query

In [94]:
wm_itm = prtl.content.get('1ad33784a4824ef38f2ce1c60b2963e0')
wm = WebMap(wm_itm)
wm.layers[0]




{
  "id": "hosted_point_layer_8467",
  "layerType": "ArcGISFeatureLayer",
  "url": "https://services.arcgis.com/dfRzIozUOVYoi65F/arcgis/rest/services/hosted_point_layer/FeatureServer/0",
  "visibility": true,
  "opacity": 1,
  "title": "hosted_point_layer",
  "itemId": "e20b714c7f8640c3a50a3636270741c7",
  "popupInfo": {
    "title": "hosted_point_layer",
    "fieldInfos": [
      {
        "fieldName": "OBJECTID",
        "label": "OBJECTID",
        "isEditable": false,
        "tooltip": "",
        "visible": false,
        "stringFieldOption": "textbox"
      },
      {
        "fieldName": "textfield",
        "label": "Text Field",
        "isEditable": true,
        "tooltip": "",
        "visible": true,
        "stringFieldOption": "textbox"
      },
      {
        "fieldName": "integerfield",
        "label": "Integer Field",
        "isEditable": true,
        "tooltip": "",
        "visible": true,
        "stringFieldOption": "textbox",
        "format": {
          "pla

### Get layer out of the webmap as a FeatureLayer

In [95]:
point_layer = FeatureLayer(wm.layers[0].url)
poly_layer = FeatureLayer(wm.layers[2].url)

### Get the story map

In [37]:
itm = prtl.content.get('876f3dc003184f2ea1d2785e294fd983')
sm = JournalStoryMap(itm)
sections = sm.properties['values']['story']['sections']

### Clear out our test section

In [122]:
# clear out any stuff we tried previously
sections[1]['content'] = ' ' 
sections[1]['contentActions'] = []
sm.save()

True

### Iterate over our features and generate the map actions

In [123]:
poly_features = poly_layer.query(out_fields='*')
point_features = point_layer.query(out_fields='*')
html_tbl = "<table><tbody><tr><th>key</th><th>value</th></tr>{0}</tbody></table>"
tbl_rows = ""
rcount = 0
for f in point_features.features:
    # Changed up the parameters a little, pass in a webmap now and a layer index
    ma = getMapAction(f, wm_itm, 0, label_field='textfield',label_prefix='',add_popup=True)
    sections[1]['contentActions'].append(ma['map_action'])
    tbl_rows += '<tr><td>{0}</td><td>{1}</td></tr>'.format(rcount, ma['content_link'])
    rcount+=1

content_tbl = html_tbl.format(tbl_rows)
print(content_tbl)
sections[1]['content'] += content_tbl
sm.save()


<table><tbody><tr><th>key</th><th>value</th></tr><tr><td>0</td><td><a data-storymaps="MJ-ACTION-620f661c-88de-4fbc-b137-eba5544300a9" data-storymaps-type="media">Charlotte</a></td></tr><tr><td>1</td><td><a data-storymaps="MJ-ACTION-3f1ee38d-8ecf-45d0-8551-a727185e22d5" data-storymaps-type="media">Springfield</a></td></tr><tr><td>2</td><td><a data-storymaps="MJ-ACTION-5070cc56-b1be-4ded-acde-933f33840d97" data-storymaps-type="media">Indianapolis</a></td></tr><tr><td>3</td><td><a data-storymaps="MJ-ACTION-f92a7de9-b6d1-4acc-ae21-10a0cce001df" data-storymaps-type="media">Norfolk</a></td></tr><tr><td>4</td><td><a data-storymaps="MJ-ACTION-fc6bf78e-456e-4d8b-a612-4c6cb9b59a5e" data-storymaps-type="media">Anchorage</a></td></tr><tr><td>5</td><td><a data-storymaps="MJ-ACTION-ba327b5b-90e7-4c37-9fb6-c651a854642a" data-storymaps-type="media">Redlands</a></td></tr><tr><td>6</td><td><a data-storymaps="MJ-ACTION-9ccf0fef-e213-4358-b138-6a4f794d5375" data-storymaps-type="media">Pierre</a></td></tr>

True